# Capstone project

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)

## Introduction: Business Problem <a name="introduction"></a>

Coyoacan is a delegation from Mexico City that is growing economically every year. In addition to this, the culture that exists in this place is attractive to encourage people to live here. Coyoacan is a place full of history and traditions, it has the neighborhoods of the town of Los Reyes, San Pedro Tepetlapa, La Candelaria, San Francisco Culhuacán, Santa Úrsula Coapa, San Francisco, El Niño Jesús, La Conchita, San Lucas, Santa Catarina. For these reasons, any company with sufficient capital should consider building apartments in this zone. 

In this project we will try to analyze which of these neighborhoods is the best option to consider investing in real estate construction. With the power of machine learning, there will be a projection of the growth of the population in this place, and therefore of the economy and the possibilities of generating profits through the collection of rents. Likewise, an attempt will be made to project through data science which neighborhood is the best option, considering that the region with the greatest number of shops and cultural places will generate the most interest.

## Data <a name="data"></a>